In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import requests
from bs4 import BeautifulSoup

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [25]:
url = 'https://finance.naver.com/item/sise_day.naver?code=005930&page=11'

In [26]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [27]:
# requests가 막혀있다
pd.read_html(res.text)

ValueError: No tables found

In [28]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(0.5)
html = driver.page_source
df = pd.read_html(html)[0]
df.dropna(inplace = True)
driver.close()

In [29]:
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.09.09,75300.0,1000.0,76400.0,76600.0,75000.0,17600770.0
2,2021.09.08,76300.0,200.0,76000.0,76400.0,75600.0,11798147.0
3,2021.09.07,76100.0,1200.0,77100.0,77100.0,75900.0,13239401.0
4,2021.09.06,77300.0,700.0,76800.0,77600.0,76600.0,12861180.0
5,2021.09.03,76600.0,600.0,76400.0,76700.0,76000.0,12096419.0
9,2021.09.02,76000.0,800.0,76800.0,76800.0,75700.0,15347486.0
10,2021.09.01,76800.0,100.0,76700.0,77100.0,75900.0,16114775.0
11,2021.08.31,76700.0,2100.0,74900.0,76700.0,74300.0,24630370.0
12,2021.08.30,74600.0,300.0,75400.0,75500.0,74200.0,12686999.0
13,2021.08.27,74300.0,300.0,74300.0,75000.0,73800.0,15172748.0


In [24]:
df_code = pd.read_csv('./data/universe_code.csv')
codes = df_code.종목코드.apply(lambda x : x[1:]).values
codes

array(['000210', '000220', '000240', '000250', '000660', '000990',
       '001340', '001390', '001500', '001750', '001800', '001820',
       '001880', '002310', '002350', '002390', '002870', '003090',
       '003220', '003410', '003530', '003550', '003670', '003780',
       '003850', '004000', '004170', '004490', '004590', '005290',
       '005670', '005690', '005880', '005930', '006060', '006220',
       '006390', '006400', '006620', '006650', '006740', '007330',
       '007340', '007570', '007700', '008490', '009150', '009420',
       '009780', '009970', '010050', '010130', '010240', '010640',
       '010780', '011040', '011170', '011330', '011780', '012330',
       '012510', '012630', '012700', '012750', '012790', '013030',
       '013120', '013810', '013890', '014570', '014680', '014830',
       '015230', '015710', '016090', '016360', '016580', '016600',
       '016610', '017040', '017510', '017890', '017960', '018250',
       '018260', '018310', '019550', '020000', '020150', '0207

In [33]:
db = MySQLdb.connect(host='localhost', port=3306, user='root', passwd='1234')
cursor = db.cursor()

In [35]:
sql = 'show databases;'
pd.read_sql(sql,db)

,Database
0,information_schema
1,budongsan
2,employees
3,fin_db
4,gmarketdb
5,mysql
6,performance_schema
7,sakila
8,shopdb
9,sqldb


In [38]:
sql = 'use fin_db;'
cursor.execute(sql)

0

In [52]:
sql = 'drop table stock_table;'
cursor.execute(sql)

0

In [53]:
sql = '''
create table if not exists stock_table (
code char(6),
date date,
close float,
margin float,
start float,
high float,
low float,
volume float);
'''

cursor.execute(sql)

0

In [55]:
url = 'https://finance.naver.com/item/sise_day.naver?code={}&page={}'
driver = webdriver.Chrome()
for code in codes[:10]:
    for i in range(1,5):
        driver.get(url.format(code, i))
        time.sleep(0.5)
        html = driver.page_source
        
        df = pd.read_html(html)[0]
        df.dropna(inplace = True)
        print(code)
        print('=' * 100)
        
        for j in range(len(df)):
            code = code
            date = df.iloc[j][0]
            close = df.iloc[j][1]
            margin = df.iloc[j][2]
            start = df.iloc[j][3]
            high = df.iloc[j][4]
            low = df.iloc[j][5]
            volume = df.iloc[j][6]
            
            sql = '''
            insert ignore into stock_table values('{}','{}', '{}', '{}', '{}', '{}', '{}', '{}');
            '''.format(code,date,close,margin,start,high,low,volume)
            
            cursor.execute(sql)
            
driver.close()    

000210
000210
000210
000210
000220
000220
000220
000220
000240
000240
000240
000240
000250
000250
000250
000250
000660
000660
000660
000660
000990
000990
000990
000990
001340
001340
001340
001340
001390
001390
001390
001390
001500
001500
001500
001500
001750
001750
001750
001750


In [56]:
sql = 'select * from stock_table;'

st_price = pd.read_sql(sql,db)
st_price

,code,date,close,margin,start,high,low,volume
0,000210,2022-02-09,60300.0,700.0,59700.0,60400.0,59000.0,12738.0
1,000210,2022-02-08,59600.0,1000.0,59100.0,61000.0,58600.0,83130.0
2,000210,2022-02-07,58600.0,1100.0,57500.0,58800.0,57000.0,53386.0
3,000210,2022-02-04,57500.0,900.0,56900.0,57800.0,56600.0,34228.0
4,000210,2022-02-03,56600.0,700.0,56300.0,57500.0,56300.0,44961.0
...,...,...,...,...,...,...,...,...
395,001750,2021-12-16,16000.0,50.0,16050.0,16100.0,15950.0,30691.0
396,001750,2021-12-15,16050.0,50.0,16000.0,16150.0,15900.0,48967.0
397,001750,2021-12-14,16100.0,100.0,15850.0,16100.0,15800.0,37002.0
398,001750,2021-12-13,16000.0,250.0,15700.0,16050.0,15700.0,36308.0


In [98]:
from datetime import datetime
st_price['date'] = pd.to_datetime(st_price['date'])
st_price[st_price['date'].dt.month == 2]

,code,date,close,margin,start,high,low,volume
0,000210,2022-02-09,60300.0,700.0,59700.0,60400.0,59000.0,12738.0
1,000210,2022-02-08,59600.0,1000.0,59100.0,61000.0,58600.0,83130.0
2,000210,2022-02-07,58600.0,1100.0,57500.0,58800.0,57000.0,53386.0
3,000210,2022-02-04,57500.0,900.0,56900.0,57800.0,56600.0,34228.0
4,000210,2022-02-03,56600.0,700.0,56300.0,57500.0,56300.0,44961.0
40,000220,2022-02-09,8620.0,1440.0,7220.0,8880.0,7200.0,9115540.0
41,000220,2022-02-08,7180.0,100.0,7250.0,7310.0,7110.0,103370.0
42,000220,2022-02-07,7280.0,80.0,7250.0,7340.0,7110.0,141024.0
43,000220,2022-02-04,7200.0,20.0,7120.0,7290.0,7080.0,160387.0
44,000220,2022-02-03,7180.0,360.0,6970.0,7600.0,6930.0,566487.0
